In [ ]:
from safetensors.torch import load_file

checkpoint = load_file("../checkpoints/svd_no_emb.safetensors")

print(checkpoint.keys())

In [ ]:
sorted([key for key in checkpoint.keys() if key.startswith("model.diffusion_model")])

In [ ]:
import sys

sys.path.append("..")
from omegaconf import OmegaConf
from sgm.util import instantiate_from_config

config = OmegaConf.load("../configs/example_training/keyframes_base.yaml")
print(config)
config["model"]["params"]["ckpt_path"] = None
video_model = instantiate_from_config(config.model)
video_model

In [ ]:
import sys

sys.path.append("..")
from omegaconf import OmegaConf
from sgm.util import instantiate_from_config

config = OmegaConf.load("../configs/example_training/keyframes_base_bad.yaml")
print(config)
config["model"]["params"]["ckpt_path"] = None
video_model_bad = instantiate_from_config(config.model)
video_model_bad

In [ ]:
video_model_bad.state_dict().keys()

In [ ]:
def compare_state_dicts(model_state_dict, checkpoint_state_dict):
    model_keys = set(k for k in model_state_dict.keys() if k.startswith("model.diffusion_model"))
    checkpoint_keys = set(k for k in checkpoint_state_dict.keys() if k.startswith("model.diffusion_model"))

    print("Keys in model but not in checkpoint:")
    for key in model_keys - checkpoint_keys:
        print(key)

    print("\nKeys in checkpoint but not in model:")
    extra_keys = checkpoint_keys - model_keys
    for key in extra_keys:
        print(key)

    return extra_keys


def adjust_checkpoint(checkpoint, extra_keys):
    modified_checkpoint = {}
    for key, value in checkpoint.items():
        if key in extra_keys:
            continue  # Skip the extra keys
        if "input_blocks" in key or "output_blocks" in key:
            parts = key.split(".")
            block_num = int(parts[3])
            if block_num > 0:
                new_block_num = block_num - 1
                new_key = ".".join(parts[:3] + [str(new_block_num)] + parts[4:])
                modified_checkpoint[new_key] = value
            else:
                modified_checkpoint[key] = value
        else:
            modified_checkpoint[key] = value
    return modified_checkpoint


# Compare the state dicts and get the extra keys
extra_keys = compare_state_dicts(video_model_bad.state_dict(), checkpoint)

# Adjust the checkpoint
adjusted_checkpoint = adjust_checkpoint(checkpoint, extra_keys)

# Compare the adjusted checkpoint with the model
compare_state_dicts(video_model_bad.state_dict(), adjusted_checkpoint)

# Compare the state dicts

In [34]:
# Create a new dictionary for the modified weights
modified_checkpoint = {}


# Create a new state dict for the modified model
modified_state_dict = {}
# Iterate through the keys in the original checkpoint
input_block_keys = set()
output_block_keys = set()

for key in checkpoint.keys():
    if key.startswith("model.diffusion_model"):
        if "input_blocks" in key:
            input_block_keys.add(key)
        elif "output_blocks" in key:
            output_block_keys.add(key)

# Count input blocks in state_bad
# Get max input block number in state_bad
state_bad_max_input_block = max(
    [
        int(key.split(".")[3])
        for key in video_model_bad.state_dict().keys()
        if key.startswith("model.diffusion_model.input_blocks")
    ]
)

checkpoint_max_input_block = max(
    [int(key.split(".")[3]) for key in checkpoint.keys() if key.startswith("model.diffusion_model.input_blocks.")]
)

# Calculate the difference in input blocks
input_block_diff = checkpoint_max_input_block - state_bad_max_input_block


# Function to adjust output block numbers
def adjust_output_block_number(key, diff):
    parts = key.split(".")
    block_index = parts.index("output_blocks") + 1
    current_block = int(parts[block_index])
    new_block = max(0, current_block - diff)
    parts[block_index] = str(new_block)
    return ".".join(parts)


# Adjust only output blocks
for key, value in checkpoint.items():
    if key.startswith("model.diffusion_model"):
        if "output_blocks" in key:
            new_key = adjust_output_block_number(key, input_block_diff)
            modified_state_dict[new_key] = value
        else:
            modified_state_dict[key] = value
    else:
        modified_state_dict[key] = value

# Print some information for verification
print(f"Input block difference (used for output block adjustment): {input_block_diff}")
print(f"Original checkpoint keys: {len(checkpoint)}")
print(f"Modified state dict keys: {len(modified_state_dict)}")

Input block difference (used for output block adjustment): 3
Original checkpoint keys: 2446
Modified state dict keys: 2377


In [36]:
# Save the modified state dict with safetensors
from safetensors.torch import save_file

save_file(modified_state_dict, "../checkpoints/svd_bad.safetensors")

In [35]:
video_model_bad.load_state_dict(modified_state_dict)

RuntimeError: Error(s) in loading state_dict for DiffusionEngine:
	Missing key(s) in state_dict: "conditioner.embedders.3.linear.weight", "conditioner.embedders.3.linear.bias". 
	Unexpected key(s) in state_dict: "model.diffusion_model.input_blocks.10.0.emb_layers.1.bias", "model.diffusion_model.input_blocks.10.0.emb_layers.1.weight", "model.diffusion_model.input_blocks.10.0.in_layers.0.bias", "model.diffusion_model.input_blocks.10.0.in_layers.0.weight", "model.diffusion_model.input_blocks.10.0.in_layers.2.bias", "model.diffusion_model.input_blocks.10.0.in_layers.2.weight", "model.diffusion_model.input_blocks.10.0.out_layers.0.bias", "model.diffusion_model.input_blocks.10.0.out_layers.0.weight", "model.diffusion_model.input_blocks.10.0.out_layers.3.bias", "model.diffusion_model.input_blocks.10.0.out_layers.3.weight", "model.diffusion_model.input_blocks.10.0.time_mixer.mix_factor", "model.diffusion_model.input_blocks.10.0.time_stack.emb_layers.1.bias", "model.diffusion_model.input_blocks

In [ ]:
state_bad = video_model_bad.state_dict()
sorted([key for key in state_bad.keys() if key.startswith("model.diffusion_model")])